In [ ]:
from sec_edgar_downloader import Downloader
import datetime
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('all')
from bs4 import BeautifulSoup
import re
import pysentiment2
import math

In [90]:
dl = Downloader("BUFN403", "sashvenk@terpmail.umd.edu")

dl.get("10-K", "MSFT")

30

In [12]:
print(pd.to_datetime("20170801163015").date())

2017-08-01


In [96]:
def get_item_7_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        raw_10k = file.read()
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    document = {}

    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            document[doc_type] = raw_10k[doc_start:doc_end]

    regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

    matches = regex.finditer(document['10-K'])
    matches = regex.finditer(document['10-K'])
    test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    test_df.columns = ['item', 'start', 'end']
    test_df['item'] = test_df.item.str.lower()
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)

    pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

    pos_dat.set_index('item', inplace=True)

    item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item8']]
    item_7_content = BeautifulSoup(item_7_raw, 'lxml')
    #print(item_1a_content.prettify()[0:1000])
    return item_7_content.get_text("\n\n")

In [97]:
def sentiment(text):
    s = SentimentIntensityAnalyzer()
    return s.polarity_scores(text)

In [114]:
sentiment(get_item_7_text("./sec-edgar-filings/MSFT/10-K/0001193125-10-171791/full-submission.txt"))

{'neg': 0.045, 'neu': 0.796, 'pos': 0.16, 'compound': 1.0}

In [111]:
sen = SentimentIntensityAnalyzer()
with open("./sec-edgar-filings/MSFT/10-K/0000891020-94-000175/full-submission.txt", 'r', encoding='utf-8') as file:
        temp = file.read()

In [112]:
s_d = sen.polarity_scores(temp)
math.tanh(math.log(s_d['pos']/s_d['neg'])/s_d['neu'])

0.9223343401174541

In [115]:
sentiment(get_item_7_text("./sec-edgar-filings/MSFT/10-K/0001193125-09-158735/full-submission.txt"))

{'neg': 0.049, 'neu': 0.792, 'pos': 0.159, 'compound': 1.0}

In [121]:
sentiment(get_item_7_text("./sec-edgar-filings/MSFT/10-K/0001193125-05-174825/full-submission.txt"))

{'neg': 0.053, 'neu': 0.812, 'pos': 0.135, 'compound': 1.0}

In [127]:
tables = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
type(tables[2]['Symbol'])

pandas.core.series.Series

In [128]:
for i in tables[2]['Symbol']:
    print(i)

AAPL
ABBV
ABT
ACN
ADBE
AIG
AMD
AMGN
AMT
AMZN
AVGO
AXP
BA
BAC
BK
BKNG
BLK
BMY
BRK.B
C
CAT
CHTR
CL
CMCSA
COF
COP
COST
CRM
CSCO
CVS
CVX
DE
DHR
DIS
DOW
DUK
EMR
F
FDX
GD
GE
GILD
GM
GOOG
GOOGL
GS
HD
HON
IBM
INTC
INTU
JNJ
JPM
KHC
KO
LIN
LLY
LMT
LOW
MA
MCD
MDLZ
MDT
MET
META
MMM
MO
MRK
MS
MSFT
NEE
NFLX
NKE
NVDA
ORCL
PEP
PFE
PG
PM
PYPL
QCOM
RTX
SBUX
SCHW
SO
SPG
T
TGT
TMO
TMUS
TSLA
TXN
UNH
UNP
UPS
USB
V
VZ
WFC
WMT
XOM
